In [ ]:
#IMPORT PACKAGE
from PIL import Image
import numpy as np
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
#from tensorflow.keras.utils import img_to_array, load_img

In [ ]:
#MAKE TRAIN AND VALIDATION DIRECTORY

#DON'T FORGET TO CHANGE THE DIRECTORY PATH
train_dir = "C:/Users/ACER/Documents/UNDIP/Dataset Tembakau resize/Training/"
train_rendah_dir = os.path.join(train_dir, "Rendah/")
train_sedang_dir = os.path.join(train_dir, "Sedang/")
train_tinggi_dir = os.path.join(train_dir, "Tinggi/")

#DON'T FORGET TO CHANGE THE DIRECTORY PATH
validation_dir = "C:/Users/ACER/Documents/UNDIP/Dataset Tembakau resize/Validation/"
validation_rendah_dir = os.path.join(validation_dir, "Rendah/")
validation_sedang_dir = os.path.join(validation_dir, "Sedang/")
validation_tinggi_dir = os.path.join(validation_dir, "Tinggi/")

In [ ]:
#SHOW TRAIN IMAGE

print("Sample rendah image:")
plt.imshow(load_img(f"{os.path.join(train_rendah_dir, os.listdir(train_rendah_dir)[0])}"))
plt.show()

print("Sample sedang image:")
plt.imshow(load_img(f"{os.path.join(train_sedang_dir, os.listdir(train_sedang_dir)[0])}"))
plt.show()

print("Sample tinggi image:")
plt.imshow(load_img(f"{os.path.join(train_tinggi_dir, os.listdir(train_tinggi_dir)[0])}"))
plt.show()

In [ ]:
# Load the first example of a Garangan Tobacco low class
sample_image  = load_img(f"{os.path.join(train_rendah_dir, os.listdir(train_rendah_dir)[0])}")

# Convert the image into its numpy array representation
sample_array = img_to_array(sample_image)

print(f"Each image has shape: {sample_array.shape}")

print(f"The maximum pixel value used is: {np.max(sample_array)}")



In [ ]:
def image_generator(train_dir,validation_dir):
    # Instantiate the ImageDataGenerator class.
    # Remember to set the rescale argument.
    train_datagen = ImageDataGenerator(rescale=1/255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode="nearest")

    # Specify the method to load images from a directory and pass in the appropriate arguments:
    # - directory: should be a relative path to the directory containing the data
    # - targe_size: set this equal to the resolution of each image (excluding the color dimension)
    # - batch_size: number of images the generator yields when asked for a next batch. Set this to 10.
    # - class_mode: How the labels are represented. Should be one of "binary", "categorical" or "sparse".
    #               Pick the one that better suits here given that the labels are going to be 1D binary labels.
    train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                        target_size=(224, 224),
                                                        batch_size=16,
                                                        class_mode="categorical")
    
    # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
    # Remember that validation data should not be augmented
    validation_datagen = ImageDataGenerator( rescale = 1/255 )

    # Pass in the appropriate arguments to the flow_from_directory method
    validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                 batch_size=16, 
                                                                 class_mode='categorical',
                                                                 target_size=(224, 224))
    ### END CODE HERE

    return train_generator,validation_generator

In [ ]:
train_generator, validation_generator = image_generator(train_dir, validation_dir)

In [ ]:
# Import the inception model  
from tensorflow.keras.applications.mobilenet import MobileNet

# Load the MobileNet model
pre_trained_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Create an instance of the inception model from the local pre-trained weights
#DON'T FORGET TO CHANGE THE DIRECTORY PATH
#local_weights_file = 'C:/Users/ACER/Downloads/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
# create_pre_trained_model

'''def create_pre_trained_model(local_weights_file):
  """
  Initializes an InceptionV3 model.
  
  Args:
    local_weights_file (string): path pointing to a pretrained weights H5 file
    
  Returns:
    pre_trained_model: the initialized InceptionV3 model
  """
  ### START CODE HERE
  pre_trained_model = InceptionV3(input_shape = (300, 300, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  # Make all the layers in the pre-trained model non-trainable
  for layer in pre_trained_model.layers:
    layer.trainable = False

  ### END CODE HERE

  return pre_trained_model

In [ ]:
#pre_trained_model = create_pre_trained_model(local_weights_file)

# Print the model summary
pre_trained_model.summary()

In [ ]:
total_params = pre_trained_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in pre_trained_model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
# output_of_last_layer

def output_of_last_layer(pre_trained_model):
  """
  Gets the last layer output of a model
  
  Args:
    pre_trained_model (tf.keras Model): model to get the last layer output from
    
  Returns:
    last_output: output of the model's last layer 
  """
  ### START CODE HERE
  last_desired_layer = pre_trained_model.get_layer('conv_pw_13_relu')
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)
  ### END CODE HERE

  return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

In [ ]:
# Print the type of the pre-trained model
print(f"The pretrained model has type: {type(pre_trained_model)}")

In [ ]:
def create_uncompiled_model():

    ### START CODE HERE
    
    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)

    ### START CODE HERE

    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(128, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)       
    # Add a final sigmoid layer for classification
    x = x = layers.Dense(3, activation='softmax')(x)         

    # Create the complete model by using the Model class
    model = Model(inputs=pre_trained_model.input, outputs=x)
    
    ### END CODE HERE

    return model

In [ ]:
# Test your uncompiled model
uncompiled_model = create_uncompiled_model()

try:
    uncompiled_model.predict(train_generator)
except:
    print("Your current architecture is incompatible with the windowed dataset, try adjusting it.")
else:
    print("Your current architecture is compatible with the windowed dataset! :)")

In [ ]:
def adjust_learning_rate(dataset):
    
    model = create_uncompiled_model()
    
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    
    ### START CODE HERE
    
    # Select your optimizer
    optimizer = tf.keras.optimizers.Adam()
    
    # Compile the model passing in the appropriate loss
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, 
                  metrics=["accuracy"]) 
    
    ### END CODE HERE
    
    history = model.fit(train_generator, epochs=20, callbacks=[lr_schedule])
    
    return history

In [ ]:
# Run the training with dynamic LR
lr_history = adjust_learning_rate(train_generator)

In [ ]:
plt.semilogx(lr_history.history["lr"], lr_history.history["loss"])
plt.axis([1e-4, 10, 0, 10])

In [ ]:
#create_final_model

def create_final_model(pre_trained_model, last_output):
  """
  Appends a custom model to a pre-trained model
  
  Args:
    pre_trained_model (tf.keras Model): model that will accept the train/test inputs
    last_output (tensor): last layer output of the pre-trained model
    
  Returns:
    model: the combined model
  """
  # SAMAKAN MODEL FINAL DENGAN MODEL UNCOMPILED
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)

  ### START CODE HERE

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = layers.Dense(128, activation='relu')(x)
  # Add a dropout rate
  x = layers.Dropout(0.2)(x)       
  # Add a final sigmoid layer for classification
  x = x = layers.Dense(3, activation='softmax')(x)         

  # Create the complete model by using the Model class
  model = Model(inputs=pre_trained_model.input, outputs=x)

  # Compile the model
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

  ### END CODE HERE
  
  return model

In [ ]:
# Save your model in a variable
model = create_final_model(pre_trained_model, last_output)

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

In [ ]:
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 100,
                    callbacks=callbacks,
                    verbose = 2)

In [ ]:
model.save('./C:/Users/ACER/Project Bangkit/model.pb')
model.save('./C:/Users/ACER/Project Bangkit/model.h5')

In [ ]:
# EXERCISE: Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = './C:/Users/ACER/Project Bangkit/'

# YOUR CODE HERE
tf.saved_model.save(model,export_dir=export_dir)

In [ ]:
model.save('./C:/Users/ACER/Project Bangkit/model')

In [ ]:
my_model = tf.keras.models.load_model('./C:/Users/ACER/Project Bangkit/model')

In [ ]:
# EXERCISE: Use the TFLiteConverter SavedModel API to initialize the converter
converter = tf.lite.TFLiteConverter.from_keras_model(my_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

In [ ]:
open('./Mbako/MyDrive/Mbako/SaveModel/Model.tflite','wb').write(tflite_model)